In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [2]:
data = pd.read_csv('../results/Networking_concepts_features.csv', engine='python')
data.head()

,ConceptA,ConceptB,ChapterDist,PageDist,LineDist,WordDist,Total,LineCount,WordCount,Complexity
0,abramson,norman,1873,5743,0,7,16,2,1,15
1,abramson,access control lists,-1791,-5009,0,0,16,0,0,15
2,abramson,access delay,3875,12861,0,0,48,0,0,13
3,abramson,access isp,44652,136996,0,0,448,0,0,-12
4,abramson,access networks,8674,15614,0,31,1056,7,1,-50


In [3]:
gt = pd.read_csv('../AL-CPL/features/proc_network_relation_v2.csv')